In [ ]:
!pip install mxnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49.1 MB 2.1 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [ ]:
from mxnet import nd
from mxnet.contrib import text

text.embedding.get_pretrained_file_names().keys()

dict_keys(['glove', 'fasttext'])

In [ ]:
print(text.embedding.get_pretrained_file_names('glove'))

['glove.42B.300d.txt', 'glove.6B.50d.txt', 'glove.6B.100d.txt', 'glove.6B.200d.txt', 'glove.6B.300d.txt', 'glove.840B.300d.txt', 'glove.twitter.27B.25d.txt', 'glove.twitter.27B.50d.txt', 'glove.twitter.27B.100d.txt', 'glove.twitter.27B.200d.txt']


#Using word vectors MODEL - Glove-twitter-25

In [ ]:
glove_27b25d = text.embedding.create('glove', pretrained_file_name='glove.twitter.27B.25d.txt')

In [ ]:
len(glove_27b25d)

1193515

algorithm used to search for synonyms by cosine similarity

In [ ]:
def knn(W, x, k):
    cos = nd.dot(W, x.reshape((-1,))) / (
        (nd.sum(W * W, axis=1) + 1e-9).sqrt() * nd.sum(x * x).sqrt())
    topk = nd.topk(cos, k=k, ret_typ='indices').asnumpy().astype('int32')
    return topk, [cos[i].asscalar() for i in topk]

In [ ]:
def get_similar_tokens(query_token, k, embed):
    topk, cos = knn(embed.idx_to_vec,
                    embed.get_vecs_by_tokens([query_token]), k+1)
    for i, c in zip(topk[1:], cos[1:]):  # Remove input words
        print('cosine sim=%.3f: %s' % (c, (embed.idx_to_token[i])))

In [ ]:
get_similar_tokens('cloud', 5, glove_27b25d)

cosine sim=0.908: system
cosine sim=0.901: technology
cosine sim=0.891: production
cosine sim=0.890: powered
cosine sim=0.883: vision


In [ ]:
def get_analogy(token_a, token_b, token_c, embed):
    vecs = embed.get_vecs_by_tokens([token_a, token_b, token_c])
    x = vecs[1] - vecs[0] + vecs[2]
    topk, cos = knn(embed.idx_to_vec, x, 1)
    return embed.idx_to_token[topk[0]]

In [ ]:
get_analogy('man', 'woman', 'son', glove_27b25d)

'son'

In [ ]:
get_analogy('beijing', 'china', 'tokyo', glove_27b25d)

'china'

In [ ]:
get_analogy('do', 'did', 'go', glove_27b25d)

'hope'

#Using word vectors MODEL - glove-wiki-gigaword-50

**(Better than the previous one)**

In [ ]:
glove_6b50d = text.embedding.create('glove', pretrained_file_name='glove.6B.50d.txt')

In [ ]:
len(glove_6b50d)

400001

In [ ]:
def knn(W, x, k):
    # The added 1e-9 is for numerical stability
    cos = nd.dot(W, x.reshape((-1,))) / (
        (nd.sum(W * W, axis=1) + 1e-9).sqrt() * nd.sum(x * x).sqrt())
    topk = nd.topk(cos, k=k, ret_typ='indices').asnumpy().astype('int32')
    return topk, [cos[i].asscalar() for i in topk]

In [ ]:
def get_similar_tokens(query_token, k, embed):
    topk, cos = knn(embed.idx_to_vec,
                    embed.get_vecs_by_tokens([query_token]), k+1)
    for i, c in zip(topk[1:], cos[1:]):  # Remove input words
        print('cosine sim=%.3f: %s' % (c, (embed.idx_to_token[i])))

In [ ]:
get_similar_tokens('chip', 3, glove_6b50d)

cosine sim=0.856: chips
cosine sim=0.749: intel
cosine sim=0.749: electronics


In [ ]:
get_similar_tokens('computer', 3, glove_6b50d)

cosine sim=0.917: computers
cosine sim=0.881: software
cosine sim=0.853: technology


In [ ]:
def get_analogy(token_a, token_b, token_c, embed):
    vecs = embed.get_vecs_by_tokens([token_a, token_b, token_c])
    x = vecs[1] - vecs[0] + vecs[2]
    topk, cos = knn(embed.idx_to_vec, x, 1)
    return embed.idx_to_token[topk[0]] 

In [ ]:
get_analogy('man', 'woman', 'son', glove_6b50d)

'daughter'

In [ ]:
get_analogy('beijing', 'china', 'tokyo', glove_6b50d)

'japan'

In [ ]:
get_analogy('bad', 'worst', 'big', glove_6b50d)

'biggest'

# Using Gensim API
(directly loading model instead of vectors)

In [ ]:
!pip install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import gensim.downloader as api

In [ ]:
model = api.load("glove-twitter-25") #change model name here to use different models
model.most_similar("glass")

[==================================================] 100.0% 104.8/104.8MB downloaded


[('lamp', 0.9047179222106934),
 ('wood', 0.9034247398376465),
 ('oz', 0.8972457051277161),
 ('sand', 0.8971636891365051),
 ('paint', 0.8952186107635498),
 ('leaf', 0.8941252827644348),
 ('bottle', 0.8934361934661865),
 ('umbrella', 0.8917711973190308),
 ('steel', 0.8896211385726929),
 ('tree', 0.8895832300186157)]

In [ ]:
POSITIVE_LIST = ['woman', 'king']
NEGATIVE_LIST = ['man']
model.most_similar(positive=POSITIVE_LIST, negative=NEGATIVE_LIST)

[('meets', 0.8841923475265503),
 ('prince', 0.832163393497467),
 ('queen', 0.8257461190223694),
 ('’s', 0.8174097537994385),
 ('crow', 0.8134994506835938),
 ('hunter', 0.8131038546562195),
 ('father', 0.811583399772644),
 ('soldier', 0.8111359477043152),
 ('mercy', 0.8082392811775208),
 ('hero', 0.8082262873649597)]